<a href="https://colab.research.google.com/github/Rdcornejov/Metodologia-para-Data-Science/blob/main/Entregable_S6_CornejoRonaldo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00


In [74]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
data = pd.read_csv('train.csv')
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
x_test = pd.read_csv('test.csv')
x_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
y_test = pd.read_csv("gender_submission.csv")
y_test.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Tratamiento de valores faltantes

Verificamos las dimensiones de los datos tanto de entrenamiento como de testeo

In [20]:
pd.DataFrame([data.shape, x_test.shape, y_test.shape], columns= ["Filas", "Columnas"], index=["data", "x_test", "y_test"])

,Filas,Columnas
data,891,12
x_test,418,11
y_test,418,2


Acomodamos la visualizacion de valores faltantes de los datos

In [21]:
pd.DataFrame([data.isnull().sum(),x_test.isnull().sum(), y_test.isnull().sum()], index = ["data", "x_test", "y_test"] ).fillna("Sin dato")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
data,0.0,0.0,0.0,0.0,0.0,177.0,0.0,0.0,0.0,0.0,687.0,2.0
x_test,0.0,Sin dato,0.0,0.0,0.0,86.0,0.0,0.0,0.0,1.0,327.0,0.0
y_test,0.0,0.0,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato


Eliminamos la variable Cabin por tener muchos valores vacíos, no aportará en el modelo

In [22]:
data.drop(['Cabin'], axis=1, inplace=True)
x_test.drop(['Cabin'], axis=1, inplace=True)

Rellenamos la variabe "Edad" por la mediana para evitar que afecten los posibles valores atipicos

In [23]:
data["Age"].fillna(data["Age"].median(), inplace=True)
x_test["Age"].fillna(x_test["Age"].median(), inplace=True)

Rellenamos por la moda en la categoria "Embarked"

In [24]:
data["Embarked"].fillna(data["Embarked"].mode()[0], inplace=True)

Rellenamos la variabe "Precio" por la mediana para evitar que afecten los posibles valores atipico

In [25]:
x_test["Fare"].fillna(x_test["Fare"].median(), inplace=True)

Ahora ya tenemos los datos completos

In [26]:
pd.DataFrame([data.isnull().sum(),x_test.isnull().sum(), y_test.isnull().sum()], index = ["data", "x_test", "y_test"] ).fillna("Sin dato")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
x_test,0.0,Sin dato,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
y_test,0.0,0.0,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato,Sin dato


## Tratamiento de variables categóricas

Seleccionamos las variables que vamos a usar tanto de los datos de entrenamiento como el de prueba

In [27]:
x_train = data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
y_train = data["Survived"]

x_test = x_test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
y_test = y_test["Survived"]

Seleccionamos las categorias que pertenece cada variable

In [28]:
nominal = ["Sex", "Embarked"]
ordinal = ["Pclass"]
numerico = ["Age", "SibSp", "Parch", "Fare"]

Las categoricas debemos pasarlo a numerico, pero se trata distinto a las nominales y ordinales. Vemos que clases ya se encuentra en números, asi que solo trataremos las categoricas nominales que son el "Sexo" y "Embarque"

In [29]:
onehotenc = OneHotEncoder(cols = nominal)

In [30]:
x_test_trans = onehotenc.fit_transform(x_test)
x_train_trans = onehotenc.fit_transform(x_train)

In [31]:
x_train_trans.head()

,Pclass,Sex_1,Sex_2,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3
0,3,1,0,22.0,1,0,7.2500,1,0,0
1,1,0,1,38.0,1,0,71.2833,0,1,0
2,3,0,1,26.0,0,0,7.9250,1,0,0
3,1,0,1,35.0,1,0,53.1000,1,0,0
4,3,1,0,35.0,0,0,8.0500,1,0,0


In [32]:
x_test_trans.head()

,Pclass,Sex_1,Sex_2,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3
0,3,1,0,34.5,0,0,7.8292,1,0,0
1,3,0,1,47.0,1,0,7.0000,0,1,0
2,2,1,0,62.0,0,0,9.6875,1,0,0
3,3,1,0,27.0,0,0,8.6625,0,1,0
4,3,0,1,22.0,1,1,12.2875,0,1,0


## Tratamiento de variables numericas (valores atípicos)

In [33]:
for col in x_train_trans[numerico]:
    print(col, "-->", stats.shapiro(x_train_trans[col]), "Se aproxima a distribucion normal" if stats.shapiro(x_train_trans[col])[1] > 0.05 else "No se aproxima a distribucion normal", "\n")

Age --> ShapiroResult(statistic=0.9541046573646538, pvalue=4.651141686041376e-16) No se aproxima a distribucion normal 

SibSp --> ShapiroResult(statistic=0.5129657630388483, pvalue=5.750830636609992e-44) No se aproxima a distribucion normal 

Parch --> ShapiroResult(statistic=0.5328139355786152, pvalue=2.3866222471723583e-43) No se aproxima a distribucion normal 

Fare --> ShapiroResult(statistic=0.5218913010396559, pvalue=1.0840444395829658e-43) No se aproxima a distribucion normal 



Como ninguna variable se aproxima a una distribución normal, utilizaremos el método de rango intercuartil.

Podemos visualizar los limites de cada variable numérica

In [39]:
df_numeric = x_train_trans[numerico]

Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

pd.DataFrame([lower_limit, upper_limit], index = ["Límite inferior", "Límite superior"]).T

,Límite inferior,Límite superior
Age,2.500,54.5000
SibSp,-1.500,2.5000
Parch,0.000,0.0000
Fare,-26.724,65.6344


Observamos la cantidad de valores atípicos de cada variable

In [45]:
outliers = (df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))
outliers.sum()

,0
Age,66
SibSp,46
Parch,213
Fare,116


Impacto de los valores atipicos en porcentaje

In [46]:
(outliers.sum() / df_numeric.shape[0]) * 100

,0
Age,7.407407
SibSp,5.162738
Parch,23.905724
Fare,13.019080


Reemplazamos por los límites

In [58]:
x_train_limpio = x_train_trans.copy()

for col in numerico:
  x_train_limpio[col] = np.where(
      x_train_limpio[col] > upper_limit[col],
      upper_limit[col],
      np.where(
          x_train_limpio[col] < lower_limit[col],
          lower_limit[col],
          x_train_limpio[col]
      )
  )

Visualizamos que ya no hay valores atipicos

In [59]:
for col in numerico:
    print(col, " -> Cantidad atípicos: ", len(x_train_limpio[(x_train_limpio[col] > upper_limit[col]) | (x_train_limpio[col] < lower_limit[col])]))

Age  -> Cantidad atípicos:  0
SibSp  -> Cantidad atípicos:  0
Parch  -> Cantidad atípicos:  0
Fare  -> Cantidad atípicos:  0


## Modelado

Normalizamos los datos tanto de prueba como de testeo antes de aplicar los modelos

In [64]:
scaler = StandardScaler()

In [65]:
x_entrenamiento = pd.DataFrame(scaler.fit_transform(x_train_limpio))
x_entrenamiento.columns = x_train_limpio.columns
x_entrenamiento

,Pclass,Sex_1,Sex_2,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3
0,0.827377,0.737695,-0.737695,-0.583432,0.810220,0.0,-0.820552,0.615838,-0.482043,-0.307562
1,-1.566107,-1.355574,1.355574,0.742685,0.810220,0.0,2.031623,-1.623803,2.074505,-0.307562
2,0.827377,-1.355574,1.355574,-0.251903,-0.602512,0.0,-0.787578,0.615838,-0.482043,-0.307562
3,-1.566107,-1.355574,1.355574,0.494038,0.810220,0.0,1.419297,0.615838,-0.482043,-0.307562
4,0.827377,0.737695,-0.737695,0.494038,-0.602512,0.0,-0.781471,0.615838,-0.482043,-0.307562
...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,0.737695,-0.737695,-0.169020,-0.602512,0.0,-0.539655,0.615838,-0.482043,-0.307562
887,-1.566107,-1.355574,1.355574,-0.832079,-0.602512,0.0,0.290823,0.615838,-0.482043,-0.307562
888,0.827377,-1.355574,1.355574,-0.086138,0.810220,0.0,-0.029155,0.615838,-0.482043,-0.307562
889,-1.566107,0.737695,-0.737695,-0.251903,-0.602512,0.0,0.290823,-1.623803,2.074505,-0.307562


In [68]:
x_testeo = pd.DataFrame(scaler.transform(x_test_trans))
x_testeo.columns = x_test_trans.columns
x_testeo

,Pclass,Sex_1,Sex_2,Age,SibSp,Parch,Fare,Embarked_1,Embarked_2,Embarked_3
0,0.827377,0.737695,-0.737695,0.452597,-0.602512,0.0,-0.792258,0.615838,-0.482043,-0.307562
1,0.827377,-1.355574,1.355574,1.488626,0.810220,0.0,-0.832765,-1.623803,2.074505,-0.307562
2,-0.369365,0.737695,-0.737695,2.731861,-0.602512,0.0,-0.701476,0.615838,-0.482043,-0.307562
3,0.827377,0.737695,-0.737695,-0.169020,-0.602512,0.0,-0.751549,-1.623803,2.074505,-0.307562
4,0.827377,-1.355574,1.355574,-0.583432,0.810220,1.0,-0.574462,-1.623803,2.074505,-0.307562
...,...,...,...,...,...,...,...,...,...,...
413,0.827377,0.737695,-0.737695,-0.169020,-0.602512,0.0,-0.781471,-1.623803,2.074505,-0.307562
414,-1.566107,-1.355574,1.355574,0.825568,-0.602512,0.0,4.145220,-1.623803,-0.482043,3.251373
415,0.827377,0.737695,-0.737695,0.784126,-0.602512,0.0,-0.820552,-1.623803,2.074505,-0.307562
416,0.827377,0.737695,-0.737695,-0.169020,-0.602512,0.0,-0.781471,-1.623803,2.074505,-0.307562


### Modelo de arboles de decisión

In [69]:
tree_model = DecisionTreeClassifier(random_state=42)

In [70]:
tree_model.fit(x_entrenamiento, y_train)

DecisionTreeClassifier(random_state=42)

In [71]:
y_pred_tree = tree_model.predict(x_testeo)

In [72]:
# Evaluar el modelo
accuracy_tree = accuracy_score(y_test, y_pred_tree)
conf_matrix_tree = confusion_matrix(y_test, y_pred_tree)
report_tree = classification_report(y_test, y_pred_tree)

print(f'Precisión del modelo: {accuracy_tree:.2f}')
print('Matriz de Confusión:')
print(conf_matrix_tree)
print('Reporte de Clasificación:')
print(report_tree)

Precisión del modelo: 0.70
Matriz de Confusión:
[[183  83]
 [ 41 111]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.82      0.69      0.75       266
           1       0.57      0.73      0.64       152

    accuracy                           0.70       418
   macro avg       0.69      0.71      0.69       418
weighted avg       0.73      0.70      0.71       418



El modelo de Árboles de Decisión presenta una precisión moderada del 70%, con un buen rendimiento en la identificación de la clase 0 (no evento), pero con dificultades para identificar correctamente la clase 1 (evento positivo), como lo indica la baja precisión de 0.57 en esta clase.

### Modelo Random Forest

In [75]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [76]:
rf_model.fit(x_entrenamiento, y_train)

RandomForestClassifier(random_state=42)

In [77]:
y_pred_rf = rf_model.predict(x_testeo)

In [78]:
# Evaluar el modelo
accuracy_tree = accuracy_score(y_test, y_pred_rf)
conf_matrix_tree = confusion_matrix(y_test, y_pred_rf)
report_tree = classification_report(y_test, y_pred_rf)

print(f'Precisión del modelo: {accuracy_tree:.2f}')
print('Matriz de Confusión:')
print(conf_matrix_tree)
print('Reporte de Clasificación:')
print(report_tree)

Precisión del modelo: 0.88
Matriz de Confusión:
[[234  32]
 [ 17 135]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.93      0.88      0.91       266
           1       0.81      0.89      0.85       152

    accuracy                           0.88       418
   macro avg       0.87      0.88      0.88       418
weighted avg       0.89      0.88      0.88       418



El modelo de Random Forest muestra un rendimiento notablemente superior, con una precisión del 88%. Este modelo mejora significativamente la identificación de ambas clases, especialmente la clase 0, con una precisión del 93% y una mejor capacidad para identificar la clase 1, con un 81% de precisión.

## Conclusión

En la comparación entre los resultados obtenidos de los modelos de Árboles de Decisión y Random Forest, se observa una clara diferencia en su rendimiento. El modelo de Árboles de Decisión alcanzó una precisión del 70%, mostrando un buen desempeño en la identificación de la clase 0 (no evento) con una precisión del 82%. Sin embargo, su capacidad para identificar la clase 1 (evento positivo) fue limitada, reflejada en una precisión de solo 57%. Por otro lado, el modelo de Random Forest demostró ser significativamente más eficaz, logrando una precisión del 88%. Este modelo no solo mejoró la identificación de la clase 0 con una precisión del 93%, sino que también incrementó la precisión de la clase 1 al 81%. Estos resultados indican que Random Forest, al ser un modelo de ensamble, maneja mejor la variabilidad de los datos y proporciona una clasificación más equilibrada, lo que lo convierte en la opción preferida para este análisis.